In [19]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sklearn
from scipy import ndimage, misc

%matplotlib inline
import cv2
import glob

img_list = []
filenamelist= []
for img in glob.glob("CCD-1/*.jpg"):
    #img_list.append(cv2.imread(img,0))
    filenamelist.append(img)
np.array(filenamelist).shape

(119,)

In [20]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center < (radius+10)
    return mask

In [21]:
def mask_innercicle(img):
    
    ret, thresh1 = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)
    _ , contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    import circle_fit as cf

    circle_point = []

    for i in range(np.array(contours).shape[0]):
            if(np.array(contours[i]).shape[0]>1500):
                #print(np.array(contours[i]).shape)

                cor = np.array(contours[i])
                cor = cor.reshape(cor.shape[0],cor.shape[2])
                #print(cor.shape)

                xc,yc,r,_ = cf.least_squares_circle(cor)
                print(xc,yc,r)
                circle_point.append([xc,yc,r])

    cor = np.array(contours[1])
    cor = cor.reshape(cor.shape[0],cor.shape[2])

    xc,yc,r,_ = cf.least_squares_circle(cor)

    h = img.shape[0]
    w = img.shape[1]
    center = (circle_point[1][0].astype(int), circle_point[1][1].astype(int))
    radius = circle_point[1][2].astype(int)
    mask = create_circular_mask(h, w, center=center, radius=radius)
    
    mask_array = np.ma.masked_array(img, mask=mask,fill_value = 255)
    img_temp = mask_array.filled()
    return img_temp

In [22]:
for iter_file in range(len(filenamelist)):
    filename = filenamelist[iter_file]
    print(filename)
    img = np.array(cv2.imread(filename,0))
    

    img_temp = mask_innercicle(img)
    
    img_temp = ndimage.minimum_filter(img_temp, size=3)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    img_temp = cv2.blur(img_temp,(3,3))
    img_temp = clahe.apply(img_temp)

    vis = img_temp.copy()
    mser = cv2.MSER_create(_delta=3, _min_area=150,_max_variation=0.25)#cv2.MSER_create(_delta=3, _min_area=100,_max_variation=0.3)
    regions = mser.detectRegions(img_temp)


    hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions[0]]
    vis = cv2.cvtColor(vis,cv2.COLOR_GRAY2RGB)

    cv2.polylines(vis, hulls, 1, (255, 0, 0))
    
    
    print(".\\Detect\\" +filenamelist[iter_file][5:-4]+'_Detect.jpg')
    cv2.imwrite(".\\Detect\\" +filenamelist[iter_file][5:-4]+'_Detect.jpg',vis)
    plt.close()

CCD-1\new (1).jpg
1115.575160661796 1087.1189412121794 760.4067512791382
1119.7895564991127 1087.0336260765619 640.8996796805218
.\Detect\\new (1)_Detect.jpg
CCD-1\new (10).jpg
1095.6433175710438 1089.5409185831072 759.4888318612321
1092.6067679451155 1088.708686627076 644.3185336131669
.\Detect\\new (10)_Detect.jpg
CCD-1\new (100).jpg
1111.2061443529408 1087.6894410345471 759.061315237736
1112.4593617183334 1088.2157164570165 643.2043602705569
.\Detect\\new (100)_Detect.jpg
CCD-1\new (103).jpg
1111.1595739892077 1090.5834608807993 759.3575340397425
1111.772237633656 1089.7918200898246 643.1669104469457
.\Detect\\new (103)_Detect.jpg
CCD-1\new (106).jpg
1104.7531247590432 1094.8335953620517 761.915620132743
1103.4201300634243 1093.763807067841 647.0299644642632
.\Detect\\new (106)_Detect.jpg
CCD-1\new (109).jpg
1120.7030527067345 1076.5650214198185 761.2592733203956
1118.973622157609 1076.8709736943686 646.9369944566693
.\Detect\\new (109)_Detect.jpg
CCD-1\new (11).jpg
1096.67834092553

.\Detect\\new (29)_Detect.jpg
CCD-1\new (3).jpg
1116.860767988204 1080.040537038347 760.4389061663207
1116.5919150991572 1080.267426581094 645.4027790055277
.\Detect\\new (3)_Detect.jpg
CCD-1\new (34).jpg
1107.557422723619 1086.8764222226364 759.9589085891702
1107.8639982603072 1084.6295297933138 644.7294952537809
.\Detect\\new (34)_Detect.jpg
CCD-1\new (39).jpg
1102.3518027173322 1096.2477802412689 760.6892133702756
1101.3685658622612 1098.48954259621 642.3500194855442
.\Detect\\new (39)_Detect.jpg
CCD-1\new (4).jpg
1106.0563269330096 1086.513971972386 760.2061950834794
1109.79726247734 1086.5228059427861 645.4525167525727
.\Detect\\new (4)_Detect.jpg
CCD-1\new (40).jpg
1095.9428235253463 1101.099089803339 758.8417414969844
1095.7617611395112 1099.3259980222012 642.6960495943084
.\Detect\\new (40)_Detect.jpg
CCD-1\new (47).jpg
1104.837698356025 1088.6287651035445 757.9481661631936
1105.9145449954726 1089.465655639529 642.6005473526305
.\Detect\\new (47)_Detect.jpg
CCD-1\new (48).jpg
1

.\Detect\\NG (34)_Detect.jpg
CCD-1\NG (35).jpg
1105.2836173780609 1100.3930933773918 759.9973609137988
1104.4155637193935 1098.792143450545 644.8819321673682
.\Detect\\NG (35)_Detect.jpg
CCD-1\NG (36).jpg
1112.0681805937954 1089.3839692696733 760.4963879006291
1115.938309400021 1089.0618464129486 645.0565667901182
.\Detect\\NG (36)_Detect.jpg
CCD-1\NG (37).jpg
1110.4503041486432 1091.7027160731272 760.0930293484031
1110.7777143559026 1089.33738192508 645.1530132198659
.\Detect\\NG (37)_Detect.jpg
CCD-1\NG (38).jpg
1112.4256297550278 1088.0975454008642 760.7646206438144
1110.6973329314978 1087.0293414304085 645.071728424058
.\Detect\\NG (38)_Detect.jpg
CCD-1\NG (39).jpg
1101.8716448046675 1093.3870755495925 761.255682604974
1103.1141841712017 1093.8346559945421 645.1566710615152
.\Detect\\NG (39)_Detect.jpg
CCD-1\NG (4).jpg
1117.9025058232705 1084.2712950253729 761.3803226578262
1116.3622763152464 1086.54750591365 645.5354037889231
.\Detect\\NG (4)_Detect.jpg
CCD-1\NG (40).jpg
1115.1636